In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
pd.set_option('display.max_columns', None)

In [178]:
df = pd.read_csv('../../data/ml/historical_clean_nan.csv')

In [4]:
lookup = pd.read_csv('../../data/ml/estacion_distance_lookup.csv')

In [5]:
df = df.round(4)

In [6]:
df.head()

,fecha,idema,tmed,prec,tmin,tmax,hr_max,hr_min,hr_media,velmedia,racha,pres_max,pres_min,sol,year,fecha_sin,fecha_cos,dir_sin,dir_cos,latitud,longitud,altitud
0,1997-01-01,3110C,0.6,0.0,-0.5,1.7,99.0,84.0,93.0,0.3,3.9,891.3,885.6,NaN,0.47,0.0172,0.9999,0.7346,0.6785,41.0000,-3.6000,1030
1,1997-01-01,1495,1.8,4.1,0.4,3.2,NaN,NaN,100.0,1.9,6.1,977.4,975.2,0.5,0.47,0.0172,0.9999,0.9096,0.4154,42.2333,-8.6167,255
2,1997-01-01,B691,10.0,6.0,4.0,16.0,NaN,NaN,84.0,0.8,6.1,NaN,NaN,2.3,0.47,0.0172,0.9999,0.2511,0.9679,39.7333,3.0000,40
3,1997-01-01,0076,7.8,6.2,4.0,11.6,NaN,NaN,81.0,4.4,11.1,1012.0,1005.8,6.7,0.47,0.0172,0.9999,0.4862,0.8738,41.2833,2.0667,4
4,1997-01-01,1024E,2.7,0.0,0.4,5.0,NaN,NaN,85.0,1.9,5.8,981.6,975.0,6.5,0.47,0.0172,0.9999,0.3717,0.9284,43.3000,-2.0333,250


In [7]:
df.isna().mean() * 100

fecha         0.000000
idema         0.000000
tmed          3.833099
prec          4.363937
tmin          5.803471
tmax          5.783674
hr_max       23.126798
hr_min       29.459082
hr_media     20.078273
velmedia     22.110905
racha        23.642751
pres_max     71.024546
pres_min     70.925431
sol          79.804806
year          0.000000
fecha_sin     0.000000
fecha_cos     0.000000
dir_sin      23.009064
dir_cos      23.009064
latitud       0.000000
longitud      0.000000
altitud       0.000000
dtype: float64

In [179]:
df = df.drop(columns=['sol'])

In [187]:
linear_impute_cols = ['tmed', 'tmin', 'tmax', 'prec', 'pres_min', 'pres_max', 'hr_media', 'hr_min', 'hr_max']
knn_impute_cols = ['velmedia', 'racha', 'dir_sin', 'dir_cos']

In [188]:
df[linear_impute_cols] = df[linear_impute_cols].interpolate(method='linear', axis=0)

In [189]:
k_neighbors=5

In [235]:
imputed_df = df.copy()[:10000]

In [239]:
imputed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fecha      10000 non-null  object 
 1   idema      10000 non-null  object 
 2   tmed       10000 non-null  float64
 3   prec       10000 non-null  float64
 4   tmin       10000 non-null  float64
 5   tmax       10000 non-null  float64
 6   hr_max     10000 non-null  float64
 7   hr_min     10000 non-null  float64
 8   hr_media   10000 non-null  float64
 9   velmedia   7765 non-null   float64
 10  racha      7511 non-null   float64
 11  pres_max   10000 non-null  float64
 12  pres_min   10000 non-null  float64
 13  year       10000 non-null  float64
 14  fecha_sin  10000 non-null  float64
 15  fecha_cos  10000 non-null  float64
 16  dir_sin    7511 non-null   float64
 17  dir_cos    7511 non-null   float64
 18  latitud    10000 non-null  float64
 19  longitud   10000 non-null  float64
 20  altitud

In [245]:
nan_idemas = imputed_df[imputed_df[knn_impute_cols].isna().any(axis=1)]

In [247]:
nan_idema_neighbors = [lookup[idema] for idema in nan_idemas['idema']]

In [250]:
nan_idema_neighbors[0][:5]

0    C426E
1    C614H
2    C455M
3    C415A
4    C635B
Name: C623I, dtype: object

In [252]:
non_nan_idemas = {}
for col in knn_impute_cols:
    non_nan_idemas[col] = imputed_df[imputed_df[col].notna()]

In [270]:
for k, v in non_nan_idemas.items():
    print(f"Nº stations that have no NaNs in {k}: {len(v)}")

Nº stations that have no NaNs in velmedia: 7765
Nº stations that have no NaNs in racha: 7511
Nº stations that have no NaNs in dir_sin: 7511
Nº stations that have no NaNs in dir_cos: 7511


As they are almost the same, except velmedia which has less NaNs, we can use any one of racha or dir to impute

In [260]:
impute_idemas = non_nan_idemas['racha']  # The rows to use for imputation

In [268]:
nan_idemas[knn_impute_cols + ['fecha', 'idema']].head()

,velmedia,racha,dir_sin,dir_cos,fecha,idema
12,NaN,NaN,NaN,NaN,1997-01-01,C623I
16,NaN,NaN,NaN,NaN,1997-01-01,C406G
19,NaN,NaN,NaN,NaN,1997-01-01,3266A
21,NaN,NaN,NaN,NaN,1997-01-01,1658
22,NaN,NaN,NaN,NaN,1997-01-01,8178D


The rows we'll be imputing

In [269]:
impute_idemas[knn_impute_cols + ['fecha', 'idema']].head()

,velmedia,racha,dir_sin,dir_cos,fecha,idema
0,0.3,3.9,0.7346,0.6785,1997-01-01,3110C
1,1.9,6.1,0.9096,0.4154,1997-01-01,1495
2,0.8,6.1,0.2511,0.9679,1997-01-01,B691
3,4.4,11.1,0.4862,0.8738,1997-01-01,0076
4,1.9,5.8,0.3717,0.9284,1997-01-01,1024E


The rows we'll be using to impute

Now we need to attach the nearest neighbor on each idema in nan_idemas